# Data-Set Overview Plots

In [ ]:
using QAOA, Distributions, Interpolations
using HDF5, Printf
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

using PyCall
np = pyimport("numpy")

PATH = "/home/ubuntu/Archives/"
PLOT_PATH = "/home/ubuntu/Archives/plots/SK_model/paper/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)

## Small-/Large-gap data comparison

In [ ]:
N = 9
N = 11
N = 13
N = 15
N = 17
N = 19

patterns_dict = Dict(
    9  => r"random_SK_instance_N_9_seed_(\d+)\.h5",
    11 => r"random_SK_instance_N_11_seed_(\d+)\.h5",
    13 => r"random_SK_instance_N_13_seed_(\d+)\.h5",
    15 => r"random_SK_instance_N_15_seed_(\d+)\.h5",
    17 => r"random_SK_instance_N_17_seed_(\d+)\.h5",
    19 => r"random_SK_instance_N_19_seed_(\d+)\.h5"
);

### Mean-Field

In [ ]:
# T_final = 32000.
T_final = 32768.
tol = 1e-6
# τ_final = 2000.
τ_final = 2048.;

In [ ]:
subdir = "small_gaps"
# subdir = "large_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name)
filter!(x -> !occursin("results", x), instance_names)
filter!(x -> !occursin("undecided", x), instance_names)
filter!(x -> !occursin("frustrated", x), instance_names)
filter!(x -> !occursin("main_df", x), instance_names);

In [ ]:
all_mf_Es = []
all_exact_E0s = []
all_exact_E1s = []
all_exact_E2s = []
all_ωs = []
spec_sum = []

ordered_seeds = []
missing_seeds = []
seed_to_energy_diff = Dict()

counter = 0
for instance_name in instance_names
    seed = match(patterns_dict[N], instance_name)[1]    
    try
        # Mean-field trajectories
        sol_t = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/times", 32768., log10(1e-6)))
        sol_u = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/trajectories", 32768., log10(1e-6)))
        nzs = reduce(hcat, [sol_u[k, 3, :] for k in 1:size(sol_u)[1]])
        
        # mean-field solution and energy
        J_mat = h5read(folder_name * instance_name, "J")
        mf_problem = Problem(0, J_mat)
        
        mf_sol = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_sol_T_final_%.0f_tol_1e%.0f", T_final, log10(tol)))
        sigma_star = sign.(mf_sol)
        h = mf_problem.local_fields
        J = mf_problem.couplings
        E_star = sum([-h[l] * sigma_star[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star[i] * sigma_star[j] for i in 1:N-1 for j in (i+1):N-1])
        push!(all_mf_Es, E_star)

        # gap and spectra
        λ = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")
        push!(all_exact_E0s, λ[1, end])
        push!(all_exact_E1s, λ[2, end])
        push!(all_exact_E2s, λ[3, end])

        # println(seed, ": " , E_star .- λ[1, end])

        gap = λ[2, :] .- λ[1, :]    
        # println(seed, ": " , 1/minimum(gap), ", ", E_star .- λ[1, end])
        seed_to_energy_diff[seed] = (E_star .- λ[1, end], E_star .- λ[1, end] / abs(λ[2, end] .- λ[1, end]))

        if isapprox(minimum(gap), 0.0, atol=0.002)
        # if isapprox(minimum(gap), 0.01, atol=0.002)
            counter += 1
        end
        push!(ordered_seeds, seed)  
    catch
        seed = match(pattern, instance_name)[1]
        push!(missing_seeds, seed)
        # printstyled(seed, "\n", color=:red)
    end
end

In [ ]:
num_plots = length(ordered_seeds)

In [ ]:
missing_seeds |> println

#### Energies

In [ ]:
ΔE = all_mf_Es .- all_exact_E0s
ΔE1 = all_mf_Es .- all_exact_E1s
ΔE2 = all_mf_Es .- all_exact_E2s
mean(ΔE) |> println
mean(ΔE1) |> println
mean(ΔE2) |> println

In [ ]:
findall(x -> isapprox(x, 0.0, atol=1e-5), ΔE) |> length |> println
findall(x -> isapprox(x, 0.0, atol=1e-5), ΔE1) |> length |> println
findall(x -> isapprox(x, 0.0, atol=1e-5), ΔE2) |> length |> println

In [ ]:
findall(x -> isapprox(x, 0.0, atol=1e-12), ΔE) |> length |> println

In [ ]:
ΔE = (all_mf_Es .- all_exact_E0s) ./ abs.(all_exact_E0s .- all_exact_E1s)
ΔE1 = (all_mf_Es .- all_exact_E1s) ./ abs.(all_exact_E0s .- all_exact_E1s)
ΔE2 = (all_mf_Es .- all_exact_E2s) ./ abs.(all_exact_E1s .- all_exact_E2s);

# ΔE = (all_mf_Es .- all_exact_E0s) ./ abs.(all_exact_E0s)
# ΔE1 = (all_mf_Es .- all_exact_E1s) ./ abs.(all_exact_E1s)
# ΔE2 = (all_mf_Es .- all_exact_E2s) ./ abs.(all_exact_E2s);

In [ ]:
num_bins = 200
num_bins = 24 # N = 9
num_bins = 32 # N = 11
num_bins = 48 # N = 13, 15
cut_off = 100
num_bins = 90 # N = 17
cutoff = 7
num_bins = 60 # N = 19
cutoff = 5

# counts, bins = np.histogram(log.(abs.(ΔE) .+ 1.0im), bins=num_bins)
# counts, bins = np.histogram(ΔE, bins=num_bins)
sorting_perm = sortperm(abs.(ΔE))[1:num_plots-cutoff]
sorted_ΔE = ΔE[sorting_perm]
counts, bins = np.histogram(sorted_ΔE[1:num_plots-cutoff], bins=num_bins)
# counts, bins = np.histogram(sort(ΔE)[1:num_plots-cutoff], bins=num_bins)
bins = real.(bins)

# counts_1, bins_1 = np.histogram(log.(abs.(ΔE1) .+ 1.0im), bins=num_bins)
# counts_1, bins_1 = np.histogram(ΔE1, bins=num_bins)
sorting_perm_1 = sortperm(abs.(ΔE1))[1:num_plots-cutoff]
sorted_ΔE1 = ΔE1[sorting_perm_1]
counts_1, bins_1 = np.histogram(sorted_ΔE1[1:num_plots-cutoff], bins=num_bins)
# counts_1, bins_1 = np.histogram(sort(ΔE1)[1:num_plots-cutoff], bins=num_bins)
bins_1 = real.(bins_1);

In [ ]:
PyPlot.rc("axes", prop_cycle=PyPlot.plt.cycler(color=["#2D5FAA", "#B7293F", "#438E6A", "#F7BD2E", "#F16C37"]))
PyPlot.rc("axes", prop_cycle=PyPlot.plt.cycler(color=["#B7293F", "#2D5FAA", "#438E6A", "#F7BD2E", "#F16C37"]))

ylims = Dict(9 => 0.8, 11 => 0.8, 13 => 0.8, 15 => 0.8, 17 => 0.8, 19 => 0.8)
xticks_dict = Dict(9 => [-1, 0, 1, 2], 11 => [-1, 0, 1, 2], 
                  13 => [-1, 0, 1, 2], 15 => [-1, 0, 1, 2], 
                  17 => [-1, 0, 1, 2], 19 => [-1, 0, 1, 2])

figure(figsize=(4, 3))

hist(bins_1[1:end-1], bins_1, weights=counts_1/length(ΔE1), alpha=0.9, label="\$E_1\$")#, label="First excited")
hist(bins[1:end-1], bins, weights=counts/length(ΔE), alpha=0.9, label="\$E_0\$")#, label="Ground")
legend(frameon=false, loc="upper left", handlelength=1.5, handletextpad=0.25)
xticks(xticks_dict[N])
xlim(xticks_dict[N][1], xticks_dict[N][end])
# xlim(-10, 10)
# ylim(0, 1.0)
ylim(0, ylims[N])
# xlabel("\$\\ln(E_* - E_i)\$")
# xlabel("\$E_* - E_i\$")
# xlabel("\$(E_* - E_i) / \\left|E_i\\right|\$")
xlabel("\$(E_* - E_{0,1}) / \\left|E_0 - E_1\\right|\$")
ylabel("\$P(E_*)\$")
tight_layout()
savefig(PLOT_PATH * @sprintf("mf_sol_hist_N_%i.pdf", N), dpi=256, bbox_inches="tight")

### Fluctuations

In [ ]:
# final time for mean-field
T_final = 32000.
# T_final = 32768.

# number of points to get Lyapunov exponent for
npts = 2048
# npts = 4096

# tolerance for DifferentialEquations.jl when solving mean-field 
tol = 1e-8;

#### Load data

##### Small gaps

In [ ]:
subdir = "small_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)

instance_names = readdir(folder_name)
# filter!(x -> occursin("results", x), instance_names)
filter!(x -> !occursin("undecided", x), instance_names)
filter!(x -> !occursin("frustrated", x), instance_names)
filter!(x -> !occursin("main_df", x), instance_names)

couplings_small_gap = Dict()
eigvals_small_gap = Dict()
lyapunov_exponents_small_gap = Dict()
bogo_spec_small_gap = Dict()
most_undecided_spins_small = Dict(zip(h5read(folder_name * @sprintf("most_undecided_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/seeds", 32768., log10(1e-6))),
                                      h5read(folder_name * @sprintf("most_undecided_spins_N_%i.h5", N), @sprintf("T_final_%.0f_tol_1e%.0f/spin_idxs", 32768., log10(1e-6)))))
most_undecided_flucs_small = Dict()

for instance_name in instance_names
    seed = match(patterns_dict[N], instance_name)[1]

    # look at results
    if occursin("results", instance_name)
        try
            # Fluctuations
            all_flucs = h5read(folder_name * instance_name, @sprintf("fluctuations_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts))
            lyapunov_exponent = sum(all_flucs, dims=1)
            if sum(lyapunov_exponent) |> abs < 1e4 # discard non-converged ones
                lyapunov_exponents_small_gap[seed] = lyapunov_exponent
                most_undecided_flucs_small[seed] = all_flucs[most_undecided_spins_small[seed], :]
            end

            # Bogoliubov spectrum
            bogo_spec = h5read(folder_name * instance_name, @sprintf("bogoliubov_spectrum_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), 32))
            bogo_spec_small_gap[seed] = bogo_spec

        catch e
            printstyled(seed, ": ", e, "\n", color=:red)
        end            
        continue
    end

    # look at instances
    try    
        couplings = h5read(folder_name * instance_name, "J")
        couplings_small_gap[seed] = couplings
        eigvals_small_gap[seed] = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")
    catch e
        printstyled(seed, ": ", e, "\n", color=:red)
    end
end

##### Large gaps

In [ ]:
subdir = "large_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)

instance_names = readdir(folder_name)
# filter!(x -> !occursin("results", x), instance_names)
filter!(x -> !occursin("undecided", x), instance_names)
filter!(x -> !occursin("frustrated", x), instance_names)
filter!(x -> !occursin("main_df", x), instance_names)

couplings_large_gap = Dict()
eigvals_large_gap = Dict()
lyapunov_exponents_large_gap = Dict()
bogo_spec_large_gap = Dict()

for instance_name in instance_names
    seed = match(patterns_dict[N], instance_name)[1]
    
    # look at results
    if occursin("results", instance_name)
        # Fluctuations
        all_flucs = h5read(folder_name * instance_name, @sprintf("fluctuations_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts))
        lyapunov_exponent = sum(all_flucs, dims=1)
        if sum(lyapunov_exponent) |> abs < 1e4 # discard non-converged ones
            lyapunov_exponents_large_gap[seed] = lyapunov_exponent
        end

        # Bogoliubov spectrum
        bogo_spec = h5read(folder_name * instance_name, @sprintf("bogoliubov_spectrum_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), 32))
        bogo_spec_large_gap[seed] = bogo_spec
        continue
    end

    # look at instances
    try
        couplings = h5read(folder_name * instance_name, "J")
        couplings_large_gap[seed] = couplings
        λ = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")
        eigvals_large_gap[seed] = λ
    catch e
        printstyled(seed, ": ", e, "\n", color=:red)
    end                                             
end

### Plots

In [ ]:
eigvals_small = [v[2, :] .- v[1, :] for (k, v) in eigvals_small_gap]
eigvals_large = [v[2, :] .- v[1, :] for (k, v) in eigvals_large_gap]
avg_eigvals_small = eigvals_small |> mean
avg_eigvals_large = eigvals_large |> mean;

In [ ]:
avg_lyapunov_exponents_small = [v for (k, v) in lyapunov_exponents_small_gap] |> mean
avg_lyapunov_exponents_large = [v for (k, v) in lyapunov_exponents_large_gap] |> mean;

In [ ]:
avg_bogo_spec_small = [v for (k, v) in bogo_spec_small_gap] |> mean
avg_bogo_spec_large = [v for (k, v) in bogo_spec_large_gap] |> mean;

In [ ]:
lyapunov_exponents_small_gap |> length |> println
lyapunov_exponents_large_gap |> length |> println

In [ ]:
navg = 32
ninterp = size(avg_lyapunov_exponents_small)[2] - 1
avg_lyapunov_exponents_small_smooth = linear_interpolation(range(0, 1, ninterp + 1)[1:end - navg + 1], moving_average(avg_lyapunov_exponents_small, navg), extrapolation_bc=Line());
avg_lyapunov_exponents_large_smooth = linear_interpolation(range(0, 1, ninterp + 1)[1:end - navg + 1], moving_average(avg_lyapunov_exponents_large, navg), extrapolation_bc=Line());

In [ ]:
PyPlot.rc("axes", prop_cycle=PyPlot.plt.cycler(color=["#2D5FAA", "#B7293F", "#438E6A", "#F7BD2E", "#F16C37"]))
ylims = Dict(9 => 0.3, 11 => 0.4, 13 => 0.4, 15 => 0.5, 17 => 0.6, 19 => 0.7)

figure(figsize=(7, 3))
ax = subplot(121)
plot(range(0, 1, 33), avg_eigvals_small, "-o", lw=2, ms=4)
plot(range(0, 1, 33), avg_eigvals_large, "-", ms=3)
plot(range(0, 1, 33)[2:end], avg_bogo_spec_small[N, :], lw=2, "--C0")
plot(range(0, 1, 33)[2:end], avg_bogo_spec_large[N, :], "--C1")
plot([], [], "-k", label="Exact")
plot([], [], "--k", label="Bogoliubov")
xlim(0, 1)
ax.set_xticks([0., 0.5, 1.0])
ylim(0, 2)
xlabel("\$s\$")
ylabel("Average Minigap")
legend(frameon=false)

ax = subplot(122)
# plot(range(0, 1, npts+1), avg_lyapunov_exponents_small[1, :], "-", ms=3)
plot(range(0, 1, npts+1), map(x -> avg_lyapunov_exponents_small_smooth(x - navg / 2ninterp), range(0, 1, npts+1)), lw=2, label="Small")
# plot(range(0, 1, npts+1), avg_lyapunov_exponents_large[1, :], "--", label="Large")
plot(range(0, 1, npts+1), map(x -> avg_lyapunov_exponents_large_smooth(x - navg / 2ninterp), range(0, 1, npts+1)), label="Large")

xlim(0, 1)
ax.set_xticks([0., 0.5, 1.0])
ylim(0, ylims[N])
xlabel("\$s\$")
ylabel("Average \$\\mathrm{Tr}\\,\\bm{F}(t, t)\$")
legend(frameon=false)
tight_layout()
savefig(PLOT_PATH * @sprintf("gap_flucs_N_%i.pdf", N), dpi=256, bbox_inches="tight")

In [ ]:
length(couplings_small_gap) |> println
length(couplings_large_gap) |> println

In [ ]:
gapsize_small_gap = []
gaplocs_small_gap = []
for (k, v) in eigvals_small_gap
    gap = v[2, :] .- v[1, :]
    minigap = minimum(gap)
    push!(gapsize_small_gap, minigap)
    exact_times = range(0, 1, 33)
    gaploc = exact_times[findfirst(x -> x == minigap, gap)]
    push!(gaplocs_small_gap, gaploc)
end 

In [ ]:
gapsize_large_gap = []
gaplocs_large_gap = []
for (k, v) in eigvals_large_gap
    gap = v[2, :] .- v[1, :]
    minigap = minimum(gap)
    push!(gapsize_large_gap, minigap)
    exact_times = range(0, 1, 33)
    gaploc = exact_times[findfirst(x -> x == minigap, gap)]
    push!(gaplocs_large_gap, gaploc)
end 

In [ ]:
figure(figsize=(7, 3))
ax = subplot(121)
hist2D(gapsize_small_gap, gaplocs_small_gap, bins=32, range=[[0, 1e-2], [0., 1.0]], cmap="gist_earth_r")
colorbar()
# colorbar(ticks=[0, 1, 2, 3])
xlim(0, 0.01)
ylim(0., 1)
xticks([0., 5e-3, 1e-2])
ax.set_xticklabels([0., "", 1e-2])
xlabel("\$\\Delta\$")
ylabel("\$s_{\\mathrm{min}}\$")

ax = subplot(122)
s_range = [0.5, 1.0]
# s_range = [0.4, 0.9]
hist2D(gapsize_large_gap, gaplocs_large_gap, bins=32, range=[s_range, [0., 1.0]], cmap="gist_earth_r")#, vmax=12)
# colorbar()
colorbar(ticks=[0, 1, 2, 3])
xlim(s_range)
# ax.set_xticks([0.1k for k in 5:10])
ax.set_xticks([0.5, 0.75, 1.0])
ylim(0.0, 1)
ax.set_yticklabels([])
xlabel("\$\\Delta\$")
# ylabel("\$s_{\\mathrm{min}}\$")
tight_layout(pad=0)
savefig(PLOT_PATH * @sprintf("gap_loc_size_N_%i.pdf", N), dpi=256, bbox_inches="tight")